In [ ]:
from vpython import *
from math import sin,cos,sqrt
import numpy as np
import matplotlib.pyplot as plt
import random

scene = canvas(title='Breathing and Bouncing Cube',background=color.white)
N = 2 # N by N by N array of atoms


k = 100
m = 10
spacing = 1
atom_radius = 0.1*spacing



dt = 0.04*(2*pi*sqrt(m/k))

class system:
    
    def __init__(self, N, atom_radius, spacing, momentumRange):
        
        self.atoms=[]
        self.springs=[]
        
        for z in range (0,N,1):
            for y in range (0,N,1):
                for x in range (0,N,1):
                    atom=sphere(pos=vector(x,y,z)*spacing, radius=atom_radius, mass=m, velocity=vector(0,6,0), texture={'file':textures.metal})
                    p = vec.random()
                    atom.velocity=vec(0,-5,0)
                    atom.mass=1
                    atom.acceleration=vec(random.randint(30,60),0,0)
                    atom.force=vector(8,0,0)
                    atom.momentum = momentumRange*p
                    atom.index=len(self.atoms)
                    self.atoms.append(atom)
                    
                    
            
        for a in self.atoms:
            for b in self.atoms:
                self.make_spring(a,b,True)
                                     
    def make_spring(self, start, end, visible):
        spring = cylinder()
        spring.pos = start.pos
        spring.color=color.red
        spring.axis = end.pos - start.pos
        spring.length = mag(spring.axis)
        spring.radius = 0.02
        spring.velocity=vector(0,0,0)
        spring.constant=100
        spring.a=spring.length
        #spring.a = random.uniform(1,1.5)
        
        #spring.b = 0.05
        spring.b= round(random.uniform(0.01,0.10),2)
        #spring.c = 1
        spring.c= random.uniform(0.1,1.5)
        
        spring.start=start
        spring.end=end
        self.springs.append(spring)
        
c = system(N, atom_radius, spacing, 0.1*spacing*sqrt(k/m))
floor = box(pos=vector(0,-5,0), size = vector(20,0.5,20),texture={'file':textures.granite},visible=True)
scene.center = floor.pos
omega = 10
g=vec(0,-9.8,0)
t=0
dt=0.05
fric_static=0.3
fric_kinetic=0.1
while t<20:
    rate(100)
    for a in c.atoms:
        a.velocity=a.velocity+g*dt
        a.pos=a.pos+a.velocity*dt
        if a.pos.y<=floor.pos.y+atom_radius*2:
            for b in c.atoms:
                b.velocity=-b.velocity
                
#                 f_up=(0,0,-k*floor.pos.z)
#                 b.acceleration=b.force/b.mass
#                 b.velocity=b.velocity+b.acceleration*dt
#                 b.pos=b.pos+b.velocity*dt
#                 fh=sqrt((b.mass*b.acceleration.x)**2 + (b.mass*b.acceleration.y)**2 ) 
#                 ff=fric_static*b.mass*g
#                 fn=b.mass*g
#                 if fn.z<0:
#                     if fh<-fn*fric_static:
#                         fh=0
#                     elif fh>=-fn*fric_static:
#                         fh=fh-fric_kinetic*fn
#                     else:
#                         True
               

    for spring in c.springs:

        #1. update all springs
        for spring2 in c.springs:           
            spring2.pos = spring2.start.pos
            spring2.axis = spring2.end.pos - spring2.start.pos
            spring2.length = mag(spring2.axis)
        
        #2. cal new length
        new_length = spring.a + (spring.b*(sin((omega*t)+spring.c)))
        F= spring.constant*(spring.length-new_length)
                
        #3. cal change
        change = new_length - spring.length
        
        #4. 1/2 change
        c2 = change/2
        
        #5. update start ball
        spring.start.pos = spring.start.pos - (spring.axis.norm()*c2)
        
        #6. update end ball
        spring.end.pos = spring.end.pos - (-spring.axis.norm()*c2)
        
        #7. update all springs
        for spring2 in c.springs:
            spring2.pos = spring2.start.pos
            spring2.axis = spring2.end.pos - spring2.start.pos
            spring2.length = mag(spring2.axis)
    
                   
            
            
    t=t+dt

